In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import sklearn
import seaborn as sns
import warnings
from sklearn.metrics import mean_squared_error
from sklearn import metrics
warnings.filterwarnings('ignore')

## Importação dos dados

In [2]:
math_train = pd.read_csv('train.csv')

In [30]:
#Redes neurais recorrentes

In [29]:
math_train.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,1,0.0,...,5.0,A,A,A,A,A,0,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,0,0.0,...,NaN,A,A,C,A,B,0,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,0,0.0,...,NaN,A,A,A,A,C,0,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,0,0.0,...,5.0,C,A,A,A,A,3,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,1,0.0,...,NaN,A,A,A,A,A,0,A,A,A


## Transformando dados categóricos

In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [4]:
def generate_labelencoder(atts):
    for attr in atts:
        math_train[attr] = le.fit_transform(math_train[attr])
    return math_train

In [5]:
math_train = generate_labelencoder(['TP_SEXO', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'])

In [6]:
math_train['TP_ENSINO'].fillna(int(math_train['TP_ENSINO'].mode()), inplace = True)
math_train['TP_DEPENDENCIA_ADM_ESC'].fillna(int(math_train['TP_DEPENDENCIA_ADM_ESC'].mode()),inplace = True)
math_train['TP_STATUS_REDACAO'].fillna(int((math_train['TP_STATUS_REDACAO']).mode()),inplace = True)

math_train['NU_NOTA_CN'].fillna(np.mean(math_train['NU_NOTA_CN']),inplace = True)
math_train['NU_NOTA_CH'].fillna(np.mean(math_train['NU_NOTA_CH']),inplace = True)
math_train['NU_NOTA_LC'].fillna(np.mean(math_train['NU_NOTA_LC']),inplace = True)
math_train['NU_NOTA_MT'].fillna(np.mean(math_train['NU_NOTA_MT']),inplace = True)
math_train['NU_NOTA_REDACAO'].fillna(np.mean(math_train['NU_NOTA_REDACAO']),inplace = True)

## Definindo Features e variável alvo

In [7]:
x = math_train[['NU_NOTA_CN','NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO','NU_IDADE','TP_ENSINO','TP_DEPENDENCIA_ADM_ESC','TP_STATUS_REDACAO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'IN_TREINEIRO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_SEXO', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']]
y = math_train['NU_NOTA_MT']

## Separação de dados de treino e teste

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

## "Encaixe" do grupo de treino no grupo de teste

In [9]:
from sklearn.linear_model import LinearRegression
reglin = LinearRegression()
reglin.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Variável para previsão

In [10]:
y_prev1 = reglin.predict(x_test)

## Teste de performance

In [11]:
#RMSE
print(np.sqrt(metrics.mean_squared_error(y_test, y_prev1)))

63.766206185795795


In [12]:
math_test = pd.read_csv('test.csv')

In [13]:
math_test['TP_ENSINO'].fillna(int(math_test['TP_ENSINO'].mode()), inplace = True)
math_test['TP_DEPENDENCIA_ADM_ESC'].fillna(int(math_test['TP_DEPENDENCIA_ADM_ESC'].mode()),inplace = True)

math_test.update(math_test['NU_NOTA_CN'].fillna(0))
math_test.update(math_test['NU_NOTA_CH'].fillna(0))
math_test.update(math_test['NU_NOTA_LC'].fillna(0))
math_test.update(math_test['NU_NOTA_REDACAO'].fillna(0))
math_test.update(math_test['TP_STATUS_REDACAO'].fillna(0))

In [14]:
def generate_labelencoder(atts):
    for attr in atts:
        math_test[attr] = le.fit_transform(math_test[attr])
    return math_test

In [15]:
math_test = generate_labelencoder(['TP_SEXO', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'])

In [16]:
features_train = ['NU_NOTA_CN','NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO','NU_IDADE','TP_ENSINO','TP_DEPENDENCIA_ADM_ESC','TP_STATUS_REDACAO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'IN_TREINEIRO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_SEXO', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']
math_test[features_train]
y_prev2 = reglin.predict(math_test[features_train])
math_test['NU_NOTA_MT'] = y_prev2

In [17]:
answer = pd.DataFrame()
answer ['NU_INSCRICAO'] = math_test.NU_INSCRICAO
answer ['NU_NOTA_MT'] = y_prev2

In [18]:
for nota in answer ['NU_NOTA_MT']:
    if nota < 0:
        answer['NU_NOTA_MT'] = answer['NU_NOTA_MT'].replace(nota, 0)

In [19]:
#answer.to_csv('answer.csv', index = False)

In [20]:
from sklearn.ensemble import RandomForestRegressor
tree_rf = RandomForestRegressor()

In [21]:
model_rf = tree_rf.fit(x_train, y_train)

In [22]:
y_prev_rf1 = model_rf.predict(x_test)

In [23]:
#MSE
mean_squared_error(y_test, y_prev_rf1)

4225.998555386873

In [24]:
#Score
model_rf.score(x_test, y_test)

0.31878647295328133

In [25]:
y_prev_rf2 = model_rf.predict(math_test[features_train])
math_test['NU_NOTA_MT'] = y_prev_rf2

In [26]:
answer = pd.DataFrame()
answer ['NU_INSCRICAO'] = math_test.NU_INSCRICAO
answer ['NU_NOTA_MT'] = y_prev_rf2

In [27]:
for nota in answer ['NU_NOTA_MT']:
    if nota < 0:
        answer['NU_NOTA_MT'] = answer['NU_NOTA_MT'].replace(nota, 0)

In [28]:
answer.to_csv('answer.csv', index = False)